# Initial Setup

In [ ]:
# This is needed only for the purpose of the notebook
!pip install ipytest

In [ ]:
# Importing required libraries
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import datasets

import pytest
import ipytest
ipytest.autoconfig()

In [ ]:
# Getting the data
iris = datasets.load_iris()

In [ ]:
# Simple setup in the data
iris_df = pd.DataFrame(iris.data, columns = iris.feature_names)
iris_df['target'] = iris.target

### Quick check in the data

In [ ]:
# Quick overview of the first 5 rows in the data
iris_df.head()

In [ ]:
# Simple counts in our target variable
iris_df['target'].value_counts()

In [ ]:
iris_df.describe()

### Setting up the classes to build a simple model

In [ ]:
class SimplePipeline:
    def __init__(self):
        self.frame = None
        # Each value is None when we instantiate the class
        self.X_train, self.X_test, self.y_train, self.Y_test = None, None, None, None
        self.model = None
        self.load_dataset()
    
    def load_dataset(self):
        """Loading the dataset, and make the train, test, split."""
        dataset = datasets.load_iris()
        
        # Removing the units (cm) from the headers
        self.feature_names = [fn[:-5] for fn in dataset.feature_names]
        self.frame = pd.DataFrame(dataset.data, columns=self.feature_names)
        self.frame['target'] = dataset.target
        
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            self.frame[self.feature_names], self.frame.target, test_size=0.65, random_state=42)
        
    def train(self, algorithm=LogisticRegression):
        
        self.model = algorithm(solver='lbfgs', multi_class='auto')
        self.model.fit(self.X_train, self.y_train)
        
    def predict(self, input_data):
        return self.model.predict(input_data)
        
    def get_accuracy(self):
        return self.model.score(X=self.X_test, y=self.y_test)
    
    def run_pipeline(self):
        """Execution method for running the pipeline several times."""
        self.load_dataset()
        self.train()

### Running the pipeline

In [ ]:
pipeline = SimplePipeline()
pipeline.run_pipeline()
accuracy_score = pipeline.get_accuracy()
print(f'The Accuracy of the model is: {accuracy_score}')

# Testing

In [ ]:
# Defining the schema
iris_schema = {
    'sepal length': {
        'range': {
            'min': 4.0,
            'max': 8.0
        },
        'dtype': float,
    },
    'sepal width': {
        'range': {
            'min': 1.0,
            'max': 5.0
        },
        'dtype': float,
    },
    'petal length': {
        'range': {
            'min': 1.0,
            'max': 7.0
        },
        'dtype': float,
    },
    'petal width': {
        'range': {
            'min': 0.1,
            'max': 3.0
        },
        'dtype': float,
    }
}

In [ ]:
@pytest.fixture
def pipeline():
    pl = SimplePipeline()
    pl.run_pipeline()
    return pl

### Creating the tests

In [ ]:
%%ipytest

def test_input_data_ranges(pipeline):
    # Getting the maximum and minimum values for each column
    max_values = pipeline.frame.max()
    min_values = pipeline.frame.min()
    
    # Ensuring that the maximum and minimum values fall into the expected range
    for feature in pipeline.feature_names:
        assert max_values[feature] <= iris_schema[feature]['range']['max']
        assert min_values[feature] >= iris_schema[feature]['range']['min']

def test_input_data_types(pipeline):
    # Getting the data types from each column
    data_types = pipeline.frame.dtypes
    
    # Testing compatibility between data types
    for feature in pipeline.feature_names:
        assert data_types[feature] == iris_schema[feature]['dtype']

# Let´s make the test fail

In [ ]:
%%ipytest


def test_input_data_ranges(pipeline):
    max_values = pipeline.frame.max()
    min_values = pipeline.frame.min()
    
    for feature in pipeline.feature_names:
        # We change the values so the new ones are not the same as those we have already in the schema
        assert max_values[feature] < 0  # This will make the test fail
        assert min_values[feature] > 1000  # This one as well

def test_input_data_types(pipeline):
    # Getting the data types from each column
    data_types = pipeline.frame.dtypes
    
    # Making comparissons between data types
    for feature in pipeline.feature_names:
        assert data_types[feature] != iris_schema[feature]['dtype']